# Networking

## HTTP con Requests

Vista l'ubiquità del web esistono varie librerie per comunicare via HTTP in Python.

Qui prenderemo in esame **Requests**:

- _"HTTP for Humans"_
- Supporto per Python 2.6–2.7 e 3.4–3.7
- Keep-Alive, HTTP Proxy, Multipart Upload, etc..

### Esempio base

In [1]:
import requests

response = requests.get('http://www.google.com')

In [2]:
response.status_code

200

In [3]:
response.text[:200]

'<!doctype html><html itemscope="" itemtype="http://schema.org/WebPage" lang="it"><head><meta content="text/html; charset=UTF-8" http-equiv="Content-Type"><meta content="/images/branding/googleg/1x/goo'

### Principali metodi

Tutti i metodi HTTP sono mappati in maniera intuitiva:

- requests.get()
- request.post()
- requests.put()
- requests.delete()
- requests.head()
- ...

### L'oggetto Response

Tutti i metodi restituiscono un oggetto di tipo **Response**.

Principali attributi:

- status_code: l'[HTTP status code][1] della risposta
- url: l'URL elaborato dalla richiesta
- text: il contenuto della risposta
- headers: gli header inviati dal server

[1]: https://httpstatuses.com/

### L'oggetto Response

L'oggetto Response offre un metodo per effettuare al volo il decoding dei contenuti in formato JSON senza passare per una libreria terza:

In [4]:
response = requests.get("https://api.github.com/feeds")

In [5]:
response.json()

{'timeline_url': 'https://github.com/timeline',
 'user_url': 'https://github.com/{user}',
 '_links': {'timeline': {'href': 'https://github.com/timeline',
   'type': 'application/atom+xml'},
  'user': {'href': 'https://github.com/{user}',
   'type': 'application/atom+xml'}}}

### Il metodo get()

Firma del metodo:

```python
get(url, params=None, **kwargs)
```

- url: l'URL della risorsa remota
- params: (opzionale) un dizionario da inviare in query string
- kwargs: altri argomenti opzionali

### Il metodo get() con parametri in query string

In [6]:
payload = {'key1': 'value1', 'key2': 'value2'}
response = requests.get('http://httpbin.org/get', params=payload)

In [7]:
response.reason

'OK'

In [8]:
response.url

'http://httpbin.org/get?key1=value1&key2=value2'

### Gestione dei redirect

Di default tutti i metodi tranne head() seguono i redirect indicati dal server remoto:

In [9]:
response = requests.get("https://www.newyorktimes.com")

In [10]:
response.status_code

200

In [11]:
response.url

'https://www.nytimes.com/'

### Gestione dei redirect

E' possibile disattivare questo comportamento impostando a _False_ l'opzione **allow_redirects**:

In [12]:
response = requests.get("https://www.newyorktimes.com", allow_redirects=False)

In [13]:
response.status_code

302

### Il metodo post()

Firma del metodo:

```python
post(url, data=None, json=None, **kwargs)
```

- url: l'URL della risorsa remota
- data: (opzionale) un dizionario di valori
- json: (opzionale) un dizionario di valori
- kwargs: altri argomenti opzionali

### Il metodo post() con dati in formato form-encoded

In [14]:
payload = {'key1': 'value1', 'key2': 'value2'}

response = requests.post("http://httpbin.org/post", data=payload)

In [15]:
response.status_code

200

In [16]:
response.json()["headers"]["Content-Type"]

'application/x-www-form-urlencoded'

### Il metodo post() con dati in formato JSON

In [17]:
payload = {'key1': 'value1', 'key2': 'value2'}

response = requests.post("http://httpbin.org/post", json=payload)

In [18]:
response.status_code

200

In [19]:
response.json()["headers"]["Content-Type"]

'application/json'

### Aggiungere degli header ad una richiesta

Per aggiungere degli header è sufficiente passarli sotto forma di dizionario al parametro __headers__:

In [ ]:
url = "https://some.remote.com/oauth/tokens"

headers = {
    "Content-Type": "application/x-www-form-urlencoded"
}

payload = {
    "client_id": "a_client_id",
    "client_secret": "the_client_secret",
    "grant_type": "client_credentials"
}

response = requests.get(url, headers=headers)

### Timeout

Di default requests non impone alcun timeout alle connessioni.

E' consigliabile impostare un timeout a tutte le richieste originate da codice di produzione tramite il parametro **timeout**:

In [30]:
response = requests.get("http://www.google.com", timeout=2)

In [31]:
response.status_code

200

### Eccezioni

Le principali eccezioni sollevate dalla libreria sono:

- __RequestException__: catchall per tutte le altre
- __Timeout__: in caso sia superato il timeout impostato
- __ConnectionError__: errore di rete (DNS, connessione rifiutata)
- __HTTPError__ (status code >= 400)

Queste e tutte le altre si trovano sotto il package _requests.exceptions_.

### Raise for status

In [33]:
response = requests.post("http://www.google.com")

In [37]:
response.raise_for_status()

HTTPError: 405 Client Error: Method Not Allowed for url: http://www.google.com/